# Regression with Boston Housing Price prediction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt`

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
print(tf.__version__)

### Boston Housing Price dataset

In [ ]:
boston_housing= keras.datasets.boston_housing

In [ ]:
data= boston_housing.load_data()
print(type(data))
print(len(data))

In [ ]:
(X_train, y_train), (X_test, y_test)= data

**Shuffling order**

In [ ]:
order= np.argsort(np.random.random(y_train.shape))
#print(order)
X_train= X_train[order]
y_train= y_train[order]

In [ ]:
X_train[0]

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                'TAX', 'PTRATIO', 'B', 'LSTAT']

In [ ]:
df= pd.DataFrame(data=X_train, columns=column_names)
df.head()

In [ ]:
print(y_train[:10])

### Feature Normalization

In [ ]:
mean= X_train.mean(axis= 0)
std= X_train.std(axis= 0)

In [ ]:
X_train= (X_train- mean)/ std
X_test= (X_test- mean)/ std

In [ ]:
print(X_train[0])

## Creating the Model

In [ ]:
def build_model():
    model= keras.Sequential([
        keras.layers.Dense(64, activation= tf.nn.relu,
                          input_shape= (X_train.shape[1],)),
        keras.layers.Dense(64, activation= tf.nn.relu),
        keras.layers.Dense(1)
    ])
    
    optimizer= tf.train.RMSPropOptimizer(0.001)
    
    model.compile(loss= 'mse',
                 optimizer= optimizer,
                 metrics= ['mae'])
    
    return model

In [ ]:
model= build_model()
model.summary()

### Train the model

In [ ]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if(epoch % 100== 0):
            print(' ')
        print('.', end= ' ')

In [ ]:
EPOCHS= 500
history= model.fit(X_train, y_train, epochs=EPOCHS,
                  validation_split= 0.2, verbose= 0,
                  callbacks= [PrintDot()])

In [ ]:
history.history.keys()

In [ ]:
def plot_history(history):
    plt.figure()
    plt.xlabel('Epoches')
    plt.ylabel('Mean Abs Error ($1000)')
    plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
            label= 'Train loss')
    plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
            label= 'Validation loss')
    plt.legend()
    plt.ylim([0, 5])
    

In [ ]:
plot_history(history)

**Early Stop**

In [ ]:
model= build_model()

early_stop= keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history= model.fit(X_train, y_train, epochs=EPOCHS,
                  validation_split= 0.2, verbose= 0,
                  callbacks= [early_stop, PrintDot()])

In [ ]:
plot_history(history)

In [ ]:
[loss, mae]= model.evaluate(X_test, y_test, verbose=1) #verbose is for showing progress bar
print('Testing Mean Abs Error ${:7.2f}'.format(mae*1000))

### Predict

In [ ]:
y_test_predict= model.predict(X_test).flatten()

plt.scatter(y_test, y_test_predict)
plt.xlabel('True value [$1000]')
plt.ylabel('Predicted value [$1000]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
plt.plot([-100, 100], [-100, 100])

In [ ]:
error= y_test_predict- y_test
plt.hist(error, bins= 50)
plt.xlabel('Prediction Error [$1000]')
plt.ylabel('Count')

## DONE